In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('ratings_final_filtered.csv')
books = pd.read_csv('books_final.csv')

In [3]:
from google.colab import userdata
MONGO_URI = userdata.get('MONGO_URI')
PINECONE_KEY = userdata.get('PINECONE_API_KEY')

In [4]:
ratings.rename(columns={'User_id':'user',"book_id":"item","rating":"label"},inplace=True)
ratings = ratings.iloc[:,[2,0,1]]

In [5]:
len(ratings)

1215645

In [6]:
books_filtered = books.dropna()
len(books_filtered)

20911

In [7]:
ratings_filtered = ratings[ratings['item'].isin(books_filtered['book_id'])]
len(ratings_filtered)

830309

In [8]:
# Initialize minified_ratings with the original DataFrame
minified_ratings = ratings_filtered.copy()

while True:
    # Count ratings for each item
    item_counts = minified_ratings['item'].value_counts()

    # Count ratings for each user
    user_counts = minified_ratings['user'].value_counts()

    # Check if there are any items with less than 2 ratings
    items_with_less_than_2 = (item_counts < 2).sum()

    # Check if there are any users with less than 2 ratings
    users_with_less_than_10 = (user_counts < 10).sum()

    # If no items or users with less than 2 ratings, stop the loop
    if items_with_less_than_2 == 0 and users_with_less_than_10 == 0:
        break

    # Filter out items with less than 2 ratings
    items_to_keep = item_counts[item_counts >= 2].index
    minified_ratings = minified_ratings[minified_ratings['item'].isin(items_to_keep)]

    # Filter out users who rated less than 2 items
    user_counts = minified_ratings['user'].value_counts()  # Recalculate after filtering items
    users_to_keep = user_counts[user_counts >= 10].index
    minified_ratings = minified_ratings[minified_ratings['user'].isin(users_to_keep)]

    # Print intermediate results
    print(f"Items with less than 2 ratings: {items_with_less_than_2}")
    print(f"Users with less than 10 ratings: {users_with_less_than_10}")
    print(f"Filtered ratings shape: {minified_ratings.shape}")

# Final result
print("Final shape of filtered ratings:", minified_ratings.shape)

Items with less than 2 ratings: 0
Users with less than 10 ratings: 492335
Filtered ratings shape: (151672, 3)
Items with less than 2 ratings: 3574
Users with less than 10 ratings: 0
Filtered ratings shape: (147280, 3)
Items with less than 2 ratings: 61
Users with less than 10 ratings: 0
Filtered ratings shape: (147210, 3)
Items with less than 2 ratings: 1
Users with less than 10 ratings: 0
Filtered ratings shape: (147209, 3)
Final shape of filtered ratings: (147209, 3)


In [9]:
len(books_filtered)

20911

In [10]:
# filter the books dataframe by checking if it is exists in filtered_ratings
minified_books = books_filtered[books_filtered['book_id'].isin(minified_ratings['item'].unique())]
len(minified_books)

15461

In [11]:
minified_books[['categories','authors']]

,categories,authors
0,['Juvenile Fiction'],['J. K. Rowling']
1,['Juvenile Fiction'],['J. R. R. Tolkien']
2,['Adventure stories'],['John Ronald Reuel Tolkien']
3,['Fiction'],['J. R. R. Tolkien']
5,['Political Science'],['Businessnews Publishing']
...,...,...
27171,['Religion'],['Dietrich Von Hildebrand']
27172,['Cooking'],['Rowan Jacobsen']
27178,['Juvenile Fiction'],['Rosie Winstead']
27179,['Fiction'],"['Lynn Morris', 'Gilbert Morris']"


In [12]:
# Function to clean categories and authors
minified_books['categories'] = minified_books['categories'].astype(str).str.replace(r"[^a-zA-Z\s]", "", regex=True)
minified_books['authors'] = minified_books['authors'].astype(str).str.replace(r"[^a-zA-Z,\s]", "", regex=True)
minified_books[['categories','authors']]

<ipython-input-12-3e546ddf4721>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minified_books['categories'] = minified_books['categories'].astype(str).str.replace(r"[^a-zA-Z\s]", "", regex=True)
<ipython-input-12-3e546ddf4721>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minified_books['authors'] = minified_books['authors'].astype(str).str.replace(r"[^a-zA-Z,\s]", "", regex=True)


,categories,authors
0,Juvenile Fiction,J K Rowling
1,Juvenile Fiction,J R R Tolkien
2,Adventure stories,John Ronald Reuel Tolkien
3,Fiction,J R R Tolkien
5,Political Science,Businessnews Publishing
...,...,...
27171,Religion,Dietrich Von Hildebrand
27172,Cooking,Rowan Jacobsen
27178,Juvenile Fiction,Rosie Winstead
27179,Fiction,"Lynn Morris, Gilbert Morris"


In [13]:
# let's upload this into the database
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.3 MB/s eta 0:00:00


In [14]:
from pymongo import MongoClient

# Replace with your MongoDB URI
client = MongoClient(MONGO_URI)

# Access your database (it will be created if it doesn't exist)
db = client['BookMatch']


In [15]:
books_record = minified_books.to_dict(orient='records')
ratings_record = minified_ratings.to_dict(orient='records')

In [16]:
# # Insert the first dataset into collection1
# collection1 = db['amazon-new']
# collection1.insert_many(books_record)

# # Insert the second dataset into collection2
# collection2 = db['ratings-new']
# collection2.insert_many(ratings_record)

# Let's create Text Embeddings and store in Pinecone vector Store

In [17]:
minified_books.columns

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'categories', 'book_id', 'ratings_count'],
      dtype='object')

In [18]:
minified_books['combined_text_features'] = books.apply(lambda row: f"{row['Title']} {row['description']} {row['categories']} {row['authors']}", axis=1)

<ipython-input-18-b7572c58943a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minified_books['combined_text_features'] = books.apply(lambda row: f"{row['Title']} {row['description']} {row['categories']} {row['authors']}", axis=1)


In [19]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 17.5 MB/s eta 0:00:00


In [20]:
import google.generativeai as genai

In [21]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [26]:
text = "Hello world"
result = genai.embed_content(model="models/text-embedding-004", content=text)

# Print just a part of the embedding to keep the output manageable
print(str(result['embedding'])[:50], '... TRIMMED]')

[0.013168523, -0.008711934, -0.046782676, 0.000699 ... TRIMMED]


In [23]:
print(len(result['embedding'])) # The embeddings have 768 dimensions

768


In [ ]:
# List to store all embeddings
all_embeddings = []

In [27]:
import pandas as pd
import time  # To introduce a delay between requests
from more_itertools import chunked  # This helps with splitting the list into chunks

# Define the chunk size (e.g., process 1000 rows at a time)
chunk_size = 1000

# Convert your DataFrame column to a list
content_list = minified_books['combined_text_features'].tolist()

# Use chunked to break the list into smaller chunks
for i, chunk in enumerate(chunked(content_list, chunk_size)):
    # Generate embeddings for the current chunk
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=chunk
    )

    # Append the embeddings of the current chunk to all_embeddings
    all_embeddings.extend(result['embedding'])

    # Optional: Print progress
    print(f"Processed chunk {i+1}/{len(content_list)//chunk_size + 1}")

    # Introduce a delay (e.g., 2 seconds) between requests
    time.sleep(2)

# Now, all_embeddings contains the final embeddings for all rows
print(f"Generated {len(all_embeddings)} embeddings")

Processed chunk 1/16
Processed chunk 2/16
Processed chunk 3/16
Processed chunk 4/16
Processed chunk 5/16
Processed chunk 6/16
Processed chunk 7/16
Processed chunk 8/16
Processed chunk 9/16
Processed chunk 10/16
Processed chunk 11/16
Processed chunk 12/16
Processed chunk 13/16
Processed chunk 14/16
Processed chunk 15/16
Processed chunk 16/16
Generated 15461 embeddings


In [28]:
# Add a new column to the DataFrame with these embeddings
minified_books['text_embeddings'] = all_embeddings

# Print the DataFrame to verify
print(minified_books[['combined_text_features', 'text_embeddings']].head())

                              combined_text_features  \
0  Harry Potter and the Sorcerers Stone Celebrate...   
1  The Hobbit Celebrating 75 years of one of the ...   
2  The Hobbit There and Back Again The adventures...   
3  The Hobbitt Or There and Back Again Illustrate...   
5  Unfit for Command Swift Boat Veterans Speak Ou...   

                                     text_embeddings  
0  [-0.038679328, -0.0021387027, -0.007678964, -0...  
1  [-0.033830225, -0.006293032, 0.05496852, 0.000...  
2  [-0.0323806, 0.025224146, 0.06751882, 0.002304...  
3  [-0.040508226, 0.010545508, 0.06477933, -0.014...  
5  [0.049009975, 0.006568412, -0.018631991, -0.02...  


<ipython-input-28-4d949c81dd05>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minified_books['text_embeddings'] = all_embeddings


In [29]:
minified_books.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,categories,book_id,ratings_count,combined_text_features,text_embeddings
0,Harry Potter and the Sorcerers Stone,Celebrate 20 years of harry potter magic! Harr...,J K Rowling,http://books.google.com/books/content?id=HksgD...,http://books.google.com/books?id=HksgDQAAQBAJ&...,Bloomsbury Publishing,2014-01-09,Juvenile Fiction,72841,3414,Harry Potter and the Sorcerers Stone Celebrate...,"[-0.038679328, -0.0021387027, -0.007678964, -0..."
1,The Hobbit,Celebrating 75 years of one of the world's mos...,J R R Tolkien,http://books.google.com/books/content?id=LLSpn...,http://books.google.com/books?id=LLSpngEACAAJ&...,Mariner Books,2012,Juvenile Fiction,73100,3343,The Hobbit Celebrating 75 years of one of the ...,"[-0.033830225, -0.006293032, 0.05496852, 0.000..."
2,The Hobbit There and Back Again,"The adventures of the well-to-do hobbit, bilbo...",John Ronald Reuel Tolkien,Not Available,http://books.google.com/books?id=6lfVzgEACAAJ&...,Unknown,1966,Adventure stories,89590,3342,The Hobbit There and Back Again The adventures...,"[-0.0323806, 0.025224146, 0.06751882, 0.002304..."
3,The Hobbitt Or There and Back Again Illustrate...,"Bilbo baggins, a respectable, well-to-do hobbi...",J R R Tolkien,http://books.google.com/books/content?id=Wbm1m...,http://books.google.nl/books?id=Wbm1mQEACAAJ&d...,Houghton Mifflin Harcourt,2013,Fiction,73099,3331,The Hobbitt Or There and Back Again Illustrate...,"[-0.040508226, 0.010545508, 0.06477933, -0.014..."
5,Unfit for Command Swift Boat Veterans Speak Ou...,The must-read summary of john e. O’neill and j...,Businessnews Publishing,http://books.google.com/books/content?id=l8koc...,http://books.google.com/books?id=l8koc6ucAp0C&...,Primento,2017-01-30,Political Science,21565,1847,Unfit for Command Swift Boat Veterans Speak Ou...,"[0.049009975, 0.006568412, -0.018631991, -0.02..."


In [30]:
minified_books.to_csv('books_with_embeddings.csv', index=False)

In [31]:
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.3 MB/s eta 0:00:00


In [32]:
from pinecone import Pinecone

In [33]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

In [34]:
# Create book index
book_index_name = 'book-text'
if book_index_name not in pc.list_indexes().names():
    pc.create_index(name=book_index_name, dimension=768, spec=ServerlessSpec(cloud="aws", region="us-east-1"))
    print(f"Index '{book_index_name}' created with dimension 768")
else:
    print(f"Index '{book_index_name}' already exists")

Index 'book-text' created with dimension 768


In [35]:
book_index = pc.Index(book_index_name)

In [36]:
book_ids = minified_books['book_id'].tolist()
embeddings = minified_books['text_embeddings'].tolist()

In [37]:
book_vectors = [{"id": str(id), "values": embedding} for id, embedding in zip(book_ids, embeddings)]

In [38]:
# Upsert in batches
def chunked_upsert(index, vectors, batch_size=500):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i + batch_size]
        index.upsert(vectors=batch)
        print(f"Upserted batch {i // batch_size + 1}")

In [39]:
chunked_upsert(book_index, book_vectors)

Upserted batch 1
Upserted batch 2
Upserted batch 3
Upserted batch 4
Upserted batch 5
Upserted batch 6
Upserted batch 7
Upserted batch 8
Upserted batch 9
Upserted batch 10
Upserted batch 11
Upserted batch 12
Upserted batch 13
Upserted batch 14
Upserted batch 15
Upserted batch 16
Upserted batch 17
Upserted batch 18
Upserted batch 19
Upserted batch 20
Upserted batch 21
Upserted batch 22
Upserted batch 23
Upserted batch 24
Upserted batch 25
Upserted batch 26
Upserted batch 27
Upserted batch 28
Upserted batch 29
Upserted batch 30
Upserted batch 31


In [44]:
def find_similar_books(book_id, top_k=10):
    query_result = book_index.query(id=str(book_id), top_k=top_k)
    # Extract the IDs of similar books from the query result
    similar_books_ids = [int(match['id']) for match in query_result['matches']]

    # Use the 'book_id' column to filter the DataFrame
    similar_books = minified_books[minified_books['book_id'].isin(similar_books_ids)]

    # Select the desired columns
    result = similar_books[['Title', 'authors', 'description', 'categories']]

    return result

In [45]:
similar_books = find_similar_books(72841)
similar_books

,Title,authors,description,categories
0,Harry Potter and the Sorcerers Stone,J K Rowling,Celebrate 20 years of harry potter magic! Harr...,Juvenile Fiction
1,The Hobbit,J R R Tolkien,Celebrating 75 years of one of the world's mos...,Juvenile Fiction
10,The Lord of the Rings Boxed Set,JRR Tolkien,"This beautiful gift edition of the hobbit, j.R...",Young Adult Fiction
21,Harry Potter and the Chamber of Secrets,J K Rowling,"Witchcraft, wizardry - fiction.",Juvenile Fiction
362,The Lion the Witch and the Wardrobe a Story fo...,C S Lewis,Narnia: a magical land full of wonder and exci...,Juvenile Fiction
526,A Wizard of Earthsea the Earthsea Cycle Book 1,Ursula K Le Guin,A boy grows to manhood while attempting to sub...,Juvenile Fiction
719,I Capture the Castle Turtleback School Library...,Dodie Smith,“every time i meet someone who also loves i ca...,Young Adult Fiction
1823,So You Want to Be a Wizard Young Wizard Series...,Cressida Cowell,Een nieuw magisch avontuur van de auteur van h...,Juvenile Fiction
18213,Harry Potter Et Le Prisonnier Dazkaban,JK Rowling,"Sirius black, le dangereux criminel qui s’est ...",Fiction
24161,Harry Potter and the Sorcerers Stone Movie Pos...,J K Rowling,In conjunction with the first live action harr...,Juvenile Fiction
